预处理数据，针对数字型异常值进行前后填充，接近连续；
不符合连续的进行中位数或众数填充

In [85]:
# 数据预处理，去除异常点
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# 这里我包装了一个异常值处理的代码，可以随便调用。
def outliers_proc(data, col_name, scale):
    """
    用于清洗异常值，默认用 box_plot（scale=3）进行清洗
    :param data: 接收 pandas 数据格式
    :param col_name: pandas 列名
    :param scale: 尺度
    :return:
    """

    def box_plot_outliers(data_ser, box_scale):
        """
        利用箱线图去除异常值
        :param data_ser: 接收 pandas.Series 数据格式
        :param box_scale: 箱线图尺度，
        :return:
        """
        iqr = box_scale * (data_ser.quantile(0.75) - data_ser.quantile(0.25))
        val_low = data_ser.quantile(0.25) - iqr
        val_up = data_ser.quantile(0.75) + iqr
        rule_low = (data_ser < val_low)
        rule_up = (data_ser > val_up)
        return (rule_low, rule_up), (val_low, val_up)
    del_idx = []
    data_n = data.copy()
    data_series = data_n[col_name]
    rule, value = box_plot_outliers(data_series, box_scale=scale)
    index = np.arange(data_series.shape[0])[rule[0] | rule[1]]
    print("Delete number is: {}".format(len(index)))
#     print('删除的索引值为：', index)
#     del_idx.append(index)
    data_n = data_n.drop(index)
    data_n.reset_index(drop=True, inplace=True)
    print("Now column number is: {}".format(data_n.shape[0]))
    index_low = np.arange(data_series.shape[0])[rule[0]]
    outliers = data_series.iloc[index_low]
    print("Description of data less than the lower bound is:")
    print(pd.Series(outliers).describe())
    index_up = np.arange(data_series.shape[0])[rule[1]]
    outliers = data_series.iloc[index_up]
    print("Description of data larger than the upper bound is:")
    print(pd.Series(outliers).describe())
    
#     fig, ax = plt.subplots(1, 2, figsize=(10, 7))
#     sns.boxplot(y=data[col_name], data=data, palette="Set1", ax=ax[0])
#     sns.boxplot(y=data_n[col_name], data=data_n, palette="Set1", ax=ax[1])
    return data_n, del_idx


def scale(features,scaler):

    scaler.fit(features)
    scaled_features=scaler.transform(features)
    return scaled_features,scaler

if __name__=='__main__':

    print('加载数据...')
    path = 'I:/old_cars/data/'
    Train_data = pd.read_csv(path + 'used_car_train_20200313.csv', sep=' ')    
    print(['原始训练数据：',Train_data.shape])
    Test_data = pd.read_csv(path + 'used_car_testA_20200313.csv', sep=' ')
    print(['测试数据：',Test_data.shape])

    df = pd.concat([Train_data, Test_data], axis=0, ignore_index=True)
    print('合并后的数据：',df.shape)
    age = []
    for i in range(len(df)):
        age0 = df.creatDate[i] - df.regDate[i]
        age1 = round((age0//10000 + (age0/1000-age0//10000*10)/12)*365, 1)
        age.append(age1)
    df['used_time'] = age
    df.drop(['regDate', 'creatDate', 'name', 'SaleID', 'seller', 'offerType'], axis=1, inplace=True)
    print('删除无效特征后数据：', df.shape)
    numeric_features = ['power', 'kilometer', 'used_time'] + ['v_{}'.format(i) for i in range(15)]
    del_index = []
    scale = [4.92, 5, 1.5, 5, 1.5, 1000, 3, 3, 1000, 1.5, 1000, 3, 3, 1.5, 6, 3, 3, 5]
    for i,col in enumerate(numeric_features):
        data, index=outliers_proc(df[~df['price'].isnull()], col, scale[i])
        
        print('各特征删除的数量：',index)
        print('箱型图删除异常值后：', data.shape)
#     data_prc.to_csv('data_prc_lstm.csv',index=None)
    print('数据预处理完毕')

加载数据...
['原始训练数据：', (150000, 31)]
['测试数据：', (50000, 30)]
合并后的数据： (200000, 31)
删除无效特征后数据： (200000, 26)
Delete number is: 223
Now column number is: 149777
Description of data less than the lower bound is:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: power, dtype: float64
Description of data larger than the upper bound is:
count      223.000000
mean      2234.515695
std       3689.001863
min        521.000000
25%        560.000000
50%        950.000000
75%       1625.500000
max      19312.000000
Name: power, dtype: float64
各特征删除的数量： []
箱型图删除异常值后： (149777, 26)
Delete number is: 0
Now column number is: 150000
Description of data less than the lower bound is:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: kilometer, dtype: float64
Description of data larger than the upper bound is:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%    

In [83]:
len(index)

12

In [69]:
del_index

[223, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 7, 1, 12]

In [86]:
# 数据预处理，去除异常点
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# 这里我包装了一个异常值处理的代码，可以随便调用。
def outliers_proc(data, col_name, scale):
    """
    用于清洗异常值，默认用 box_plot（scale=3）进行清洗
    :param data: 接收 pandas 数据格式
    :param col_name: pandas 列名
    :param scale: 尺度
    :return:
    """

    def box_plot_outliers(data_ser, box_scale):
        """
        利用箱线图去除异常值
        :param data_ser: 接收 pandas.Series 数据格式
        :param box_scale: 箱线图尺度，
        :return:
        """
        iqr = box_scale * (data_ser.quantile(0.75) - data_ser.quantile(0.25))
        val_low = data_ser.quantile(0.25) - iqr
        val_up = data_ser.quantile(0.75) + iqr
        rule_low = (data_ser < val_low)
        rule_up = (data_ser > val_up)
        return (rule_low, rule_up), (val_low, val_up)
    data_n = data.copy()
    data_series = data_n[col_name]
    rule, value = box_plot_outliers(data_series, box_scale=scale)
    index = np.arange(data_series.shape[0])[rule[0] | rule[1]]
    print("Delete number is: {}".format(len(index)))
#     print('删除的索引值为：', index)
#     del_idx.append(index)
    data_n = data_n.drop(index)
    data_n.reset_index(drop=True, inplace=True)
    print("Now column number is: {}".format(data_n.shape[0]))
    index_low = np.arange(data_series.shape[0])[rule[0]]
    outliers = data_series.iloc[index_low]
    print("Description of data less than the lower bound is:")
    print(pd.Series(outliers).describe())
    index_up = np.arange(data_series.shape[0])[rule[1]]
    outliers = data_series.iloc[index_up]
    print("Description of data larger than the upper bound is:")
    print(pd.Series(outliers).describe())
    
#     fig, ax = plt.subplots(1, 2, figsize=(10, 7))
#     sns.boxplot(y=data[col_name], data=data, palette="Set1", ax=ax[0])
#     sns.boxplot(y=data_n[col_name], data=data_n, palette="Set1", ax=ax[1])
    return data_n, del_idx


def scale(features,scaler):

    scaler.fit(features)
    scaled_features=scaler.transform(features)
    return scaled_features,scaler

if __name__=='__main__':

    print('加载数据...')
    path = 'I:/old_cars/data/'
    df = pd.read_csv(path + 'used_car_train_20200313.csv', sep=' ')    
    print(['原始训练数据：',Train_data.shape])
#     Test_data = pd.read_csv(path + 'used_car_testA_20200313.csv', sep=' ')
#     print(['测试数据：',Test_data.shape])

#     df = pd.concat([Train_data, Test_data], axis=0, ignore_index=True)
#     print('合并后的数据：',df.shape)
    age = []
    for i in range(len(df)):
        age0 = df.creatDate[i] - df.regDate[i]
        age1 = round((age0//10000 + (age0/1000-age0//10000*10)/12)*365, 1)
        age.append(age1)
    df['used_time'] = age
    df.drop(['regDate', 'creatDate', 'name', 'SaleID', 'seller', 'offerType'], axis=1, inplace=True)
    print('删除无效特征后数据：', df.shape)
    numeric_features = ['power', 'kilometer', 'used_time'] + ['v_{}'.format(i) for i in range(15)]
    del_index = []
    scale = [4.92, 5, 1.5, 5, 1.5, 1000, 3, 3, 1000, 1.5, 1000, 3, 3, 1.5, 6, 3, 3, 5]
    for i,col in enumerate(numeric_features):
        data, index=outliers_proc(df, col, scale[i])
        df = data
        print('各特征删除的数量：',index)
        print('箱型图删除异常值后：', df.shape)
#     data_prc.to_csv('data_prc_lstm.csv',index=None)
    print('数据预处理完毕')

加载数据...
['原始训练数据：', (150000, 31)]
删除无效特征后数据： (150000, 26)
Delete number is: 223
Now column number is: 149777
Description of data less than the lower bound is:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: power, dtype: float64
Description of data larger than the upper bound is:
count      223.000000
mean      2234.515695
std       3689.001863
min        521.000000
25%        560.000000
50%        950.000000
75%       1625.500000
max      19312.000000
Name: power, dtype: float64
各特征删除的数量： []
箱型图删除异常值后： (149777, 26)
Delete number is: 0
Now column number is: 149777
Description of data less than the lower bound is:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: kilometer, dtype: float64
Description of data larger than the upper bound is:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: kilometer, dtype: f